In [ ]:
import os
import pandas as pd
from datetime import datetime

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', 1000)



In [ ]:
file_list = sorted(os.listdir('datalib/mdecustom/'), reverse=True)
print("Current Rule List: " , file_list[1])
print("Prior Rule List: " , file_list[2])

In [ ]:
current_data_df = pd.read_csv('datalib/mdecustom/' + str(file_list[1]), index_col='id')
prior_data_df = pd.read_csv('datalib/mdecustom/' + str(file_list[2]), index_col='id')

## Check the Rule Count

> - This check will let you know if you need to validate any adds or removes

In [ ]:
print("Current Rule Count: " + str(len(current_data_df)))
print("Prior Rule Count: " + str(len(prior_data_df)))

## Look at Rule Changes Last 14 Days

In [ ]:
current_data_df['lastUpdatedTime'] = pd.to_datetime(current_data_df['lastUpdatedTime'])
today = datetime.utcnow().timetuple().tm_yday
current_data_df['doy'] = current_data_df['lastUpdatedTime'].dt.dayofyear
change_df = current_data_df.loc[today - current_data_df['doy'] <= 14]
print("Rule Changes Last 14 Days")
print("###########################")
change_df[['lastUpdatedTime' , 'ruleName']]

In [ ]:
current_data_df['creationTime'] = pd.to_datetime(current_data_df['creationTime'])
today = datetime.utcnow().timetuple().tm_yday
current_data_df['doy'] = current_data_df['creationTime'].dt.dayofyear
current_data_df['year'] = current_data_df['creationTime'].dt.year
new_df = current_data_df.loc[today - current_data_df['doy'] <= 14]
new_df = new_df.loc[new_df['year'] == datetime.now().year]
print("Rule Creations Last 14 Days")
print("###########################")
new_df[['creationTime' , 'ruleName']]

## Diff Checks

In [ ]:
diff_a_df = current_data_df[['ruleName', 'alertTitle', 'queryText', 'isEnabled', 'alertSeverity', 'alertCategory', 'alertSeverity', 'mitreTechniques']]
diff_b_df = prior_data_df[['ruleName', 'alertTitle', 'queryText', 'isEnabled', 'alertSeverity', 'alertCategory', 'alertSeverity', 'mitreTechniques']]

combo_df = pd.concat([diff_a_df, diff_b_df])
diff_df = combo_df.drop_duplicates(keep=False)

diff_df

## Config Check
> - Check Enabled
> - Check Informational
> - Alert Title not starts with SOC

In [ ]:
current_disabled_df = current_data_df.loc[current_data_df['isEnabled'] == False]
print("Current Disabled Count: " + str(len(current_disabled_df)))

prior_disabled_df = prior_data_df.loc[prior_data_df['isEnabled'] == False]
print("Prior Disabled Count: " + str(len(prior_disabled_df)))

print("Rule Set to Disabled")
print("#######################")
current_disabled_df[['ruleName', 'lastUpdatedTime']]

In [ ]:
current_info_df = current_data_df.loc[current_data_df['alertSeverity'] == 'Informational']
print("Current Info Count: " + str(len(current_info_df)))

prior_info_df = prior_data_df.loc[prior_data_df['alertSeverity'] == 'Informational']
print("Prior Info Count: " + str(len(prior_info_df)))

print("Rule Set to Informational")
print("#######################")
current_info_df[['ruleName', 'lastUpdatedTime']]

In [ ]:
current_soc_df = current_data_df.loc[(current_data_df['alertTitle'].str.startswith('SOC')) |
                                     (current_data_df['alertTitle'].str.startswith('SIRT'))
                                     == False]
print("Current Info Count: " + str(len(current_soc_df)))

prior_soc_df = prior_data_df.loc[(prior_data_df['alertTitle'].str.startswith('SOC')) |
                                 (prior_data_df['alertTitle'].str.startswith('SIRT'))
                                 == False]
print("Prior SOC Count: " + str(len(prior_soc_df)))

print("Not Going To SOC By Name")
print("#######################")
current_soc_df[['ruleName', 'lastUpdatedTime']]